## Load necessary libraries

In [1]:
import pickle 
embeddings, labels, seq_ids = pickle.load(open("LTRBERT_LTR_classifier_embeddings.b_average", "rb"))

In [2]:
import numpy as np
neg, pos = np.unique(labels, return_counts=True)[1]

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, random_state=42, test_size=0.1)

In [11]:
weight_for_0 = (1 / neg) * (len(labels) / 2.0)
weight_for_1 = (1 / pos) * (len(labels) / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}

In [70]:
import tensorflow as tf
import keras
import numpy as np
nn = keras.models.Sequential()
nn.add(keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(768,1)))
nn.add(keras.layers.MaxPooling1D(pool_size=2))
nn.add(keras.layers.Flatten())
nn.add(keras.layers.Dense(units=128, activation='relu'))
nn.add(keras.layers.Dropout(0.5)) 
nn.add(keras.layers.Dense(units=1, activation='sigmoid'))

nn.compile(loss='binary_crossentropy', optimizer=keras.optimizers.AdamW(), metrics=['binary_accuracy'], weighted_metrics=['binary_accuracy'])

nn.fit(X_train, np.array(y_train), epochs=10, batch_size=16, validation_data=(X_test, y_test), class_weight=class_weight, callbacks=[keras.callbacks.EarlyStopping(patience=3)])

Epoch 1/10
5618/5618 [==============================] - 16s 3ms/step - loss: 0.4700 - binary_accuracy: 0.7852 - weighted_binary_accuracy: 0.7852 - val_loss: 0.4345 - val_binary_accuracy: 0.7988 - val_weighted_binary_accuracy: 0.7988
Epoch 2/10
5618/5618 [==============================] - 15s 3ms/step - loss: 0.4458 - binary_accuracy: 0.7962 - weighted_binary_accuracy: 0.7962 - val_loss: 0.4191 - val_binary_accuracy: 0.8043 - val_weighted_binary_accuracy: 0.8043
Epoch 3/10
5618/5618 [==============================] - 15s 3ms/step - loss: 0.4368 - binary_accuracy: 0.8025 - weighted_binary_accuracy: 0.8025 - val_loss: 0.4205 - val_binary_accuracy: 0.8120 - val_weighted_binary_accuracy: 0.8120
Epoch 4/10
5618/5618 [==============================] - 15s 3ms/step - loss: 0.4324 - binary_accuracy: 0.8048 - weighted_binary_accuracy: 0.8048 - val_loss: 0.4225 - val_binary_accuracy: 0.8084 - val_weighted_binary_accuracy: 0.8084
Epoch 5/10
5618/5618 [==============================] - 14s 3ms/step

: 

In [69]:
nn.save('CNN_BERT_pool_model')

INFO:tensorflow:Assets written to: CNN_BERT_pool_model/assets


INFO:tensorflow:Assets written to: CNN_BERT_pool_model/assets


## Run predictions on test set 

In [7]:
test_embeddings, test_labels, seq_ids = pickle.load(open("/var/tmp/xhorvat9/ltr_bert/NewClassifiers/LTR_classifier/BERT/LTRBERT_LTR_classifier_embeddings_TEST.b_average", "rb"))

In [8]:
#test_embeddings, test_labels = pickle.load(open("/var/tmp/xhorvat9/ltr_bert/NewClassifiers/Superfamily/BERT/LTRBERT_superfamily_classifier_embeddings_TEST.b_average", "rb"))
predictions = nn.predict(test_embeddings)
pred = np.array([1 if x > 0.5 else 0 for x in predictions])

1823/1823 [==============================] - 2s 844us/step


In [9]:
np.array([1 if x > 0.5 else 0 for x in nn.predict(test_embeddings)])

1823/1823 [==============================] - 2s 823us/step


array([1, 1, 1, ..., 0, 0, 0])

In [10]:
from sklearn.metrics import classification_report

classification_report(np.array(test_labels), np.array([1 if x > 0.5 else 0 for x in nn.predict(test_embeddings)]), output_dict=True)

1823/1823 [==============================] - 1s 802us/step


{'0': {'precision': 0.92022433882899,
  'recall': 0.8302652242191589,
  'f1-score': 0.8729332446625542,
  'support': 22924.0},
 '1': {'precision': 0.8965847176079734,
  'recall': 0.9533687542392042,
  'f1-score': 0.9241052472982784,
  'support': 35384.0},
 'accuracy': 0.9049701584688208,
 'macro avg': {'precision': 0.9084045282184817,
  'recall': 0.8918169892291816,
  'f1-score': 0.8985192459804163,
  'support': 58308.0},
 'weighted avg': {'precision': 0.905878719749542,
  'recall': 0.9049701584688208,
  'f1-score': 0.9039867903383185,
  'support': 58308.0}}